# MLP Head

MLP HeadはViT分類期。
Encoderで出力されたデータをLayer Normalizationして線形層を通すだけ。

## 数式表現

$z_L$ を$L$個のEncoder Blockで処理したデータとする。この中からクラストークン$z_L^{cls} \in \mathbb{R}^{D}$のみ抜き取る。


これにLayer Normalizationを適用するので$LN(z_L^{cls})$となる。さらに線形層$W^y \in \mathbb{R}^{D \times M}$である。但し$M$は分類するクラス数である。

したがってMLP Head出の処理は次のように表現できる。

$$z_L \rightarrow LN(z_L^{cls}) W^y \in \mathbb{R}^{M} $$






In [7]:
import torch
import torch.nn as nn

class VitMLPHeader(nn.Module):

    def __init__(
        self,
        emb_dim:int = 384,
        class_num: int = 10
    ):
        """

        Args:
            emb_dim (int, optional): 埋め込みベクトルの長さ. Defaults to 384.
            class_num (int, optional): 分類するクラス数. Defaults to 10.
        """
        super().__init__()
        self.mlp_header = nn.Sequential(
            nn.LayerNorm(emb_dim),
            nn.Linear(emb_dim, class_num)
        )

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        """

        Args:
            z (torch.Tensor): Encoder後のデータ。(B, N, D)
                B: バッチ数
                N: トークン数
                D: 埋め込みベクトルの長さ

        Returns:
            torch.Tensor: MLP Headerの出力。(B, N, M)
                B: バッチ数
                N: トークン数
                M: 分類するクラス数 
        """
        # クラストークンのみ抜き取る
        ## (B, N, D) -> (B, D)
        cls_token = z[:, 0]

        # MLP Header
        ## (B, D) -> (B, M)
        out = self.mlp_header(cls_token)

        return out


mlp_header = VitMLPHeader()
z = mlp_header.forward(z_0)

print(z.shape)


torch.Size([2, 10])


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# emb: 埋め込み。embedded

class VitInputLayer(nn.Module):
    def __init__(
        self,
        in_channels: int = 3,
        emb_dim: int = 384,
        num_patch_row: int = 2,
        image_size: int = 32
    ):
        """        
        Args:
            in_channels (int, optional): 入力画像のチャンネル数. Defaults to 3.
            emb_dim (int, optional): 埋め込み後のベクトルの長さ. Defaults to 384.
            num_patch_row (int, optional): 高さ方向のパッチ数. Defaults to 2.
            image_size (int, optional): 入力画像の1辺の大きさ. Defaults to 32.
        """
        super().__init__()
        self.in_channels = in_channels
        self.emb_dim = emb_dim
        self.num_patch_row = num_patch_row
        self.image_size = image_size

        # パッチ数
        self.num_patch = self.num_patch_row ** 2

        # パッチの大きさ
        self.patch_size = int(self.image_size // self.num_patch_row)

        # 乳画像のパッチへの分割 & パッチ埋め込みを一気に行う層
        self.patch_emb_layer = nn.Conv2d(
            in_channels=self.in_channels,
            out_channels=self.emb_dim,
            kernel_size=self.patch_size,
            stride=self.patch_size
        )

        # クラストークン
        self.cls_token = nn.parameter.Parameter(
                torch.randn(1,1, emb_dim)
            
        )

        # 位置埋め込み
        # トークン数(パッチ数+クラストークン数(1))
        num_token = self.patch_size + 1
        self.pos_emb = nn.parameter.Parameter(
            torch.randn(1, self.num_patch+1, emb_dim)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """前処理

        Args:
            x (torch.Tensor): （B, C, H, W）
                B: バッチサイズ
                C: チャンネル数
                H: 高さ
                W: 幅

        Returns:
            torch.Tensor: ViTへの入力。(B, N, D)
                B: バッチサイズ
                N: トークン数
                D: 埋め込みベクトルの長さ
        """

        # パッチの埋め込み

        ## P: パッチの1辺のサイズ
        ## flattenはパッチ埋め込みの後

        ## パッチ埋め込み (B, C, H, W) -> (B, D, H/P, W/P)
        z_0 = self.patch_emb_layer(x)

        ## flatten (B, D, H/P, W/P) -> (B, D, Np)
        ## Np はパッチ数 (= H*W / P**2)
        z_0 = z_0.flatten(2)

        ## 軸の順番を変更 (B, D, Np) -> (B, Np, D)
        z_0 = z_0.transpose(1, 2)

        # クラストークンを結合 (B, Np, D) -> (B, N, D)
        # cls_token: (1, 1, D) から (B, 1, D)に変換して結合
        z_0 = torch.cat(
            [self.cls_token.repeat(repeats=(x.size(0), 1, 1)), z_0],
            dim=1
        )

        # 位置埋め込み
        z_0 = z_0 + self.pos_emb

        return z_0

class MultiHeadSelfAttention(nn.Module):
    def __init__(
        self,
        emb_dim: int = 384,
        head: int = 3,
        dropout: float = 0
    ):
        """

        Args:
            emb_dim (int, optional): Input Layerから出てくるベクトルの次元. Defaults to 384.
            head (int, optional): ヘッドの数. Defaults to 3.
            dropout (float, optional): ドロップアウト数. Defaults to 0.
        """
        super().__init__()
 
        self.emb_dim = emb_dim
        self.head = head
        self.head_dim = emb_dim // head
        self.sqrt_dh = self.head_dim ** 0.5 # softmax関数の時に使う
    
        # q,k,vの線形層
        self.w_q = nn.Linear(emb_dim, emb_dim, bias=False)
        self.w_k = nn.Linear(emb_dim, emb_dim, bias=False)
        self.w_v = nn.Linear(emb_dim, emb_dim, bias=False)

        # 正規化するときにDropoutをする
        self.attn_drop = nn.Dropout(dropout)

        # MHSAの結果を出力に埋め込むための線形層
        self.w_o = nn.Sequential(
            nn.Linear(emb_dim, emb_dim),
            nn.Dropout(dropout)
        )

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        """

        Args:
            z (torch.Tensor): MHSAの入力。(B, N, D)
                B: バッチ数
                N: トークン数
                D: ベクトルの次元
            

        Returns:
            torch.Tensor: MHSAの出力。(B, N, D)
                B: バッチ数
                N: トークン数
                D: ベクトルの次元
        """
        
        batch_num, patch_num, _ = z.size()

        # 埋め込み
        ## (B, N, D) -> (B, N, D)
        q = self.w_q(z)
        k = self.w_k(z)
        v = self.w_v(z)

        # ヘッドを分割
        ## h: ヘッド数
        ## (B, N, D) -> (B, N, h, D/h)
        q = q.view(batch_num, patch_num, self.head, self.head_dim)
        k = q.view(batch_num, patch_num, self.head, self.head_dim)
        v = q.view(batch_num, patch_num, self.head, self.head_dim)

        # transposeして (B, N, h, D/h) から (B, h, N, D/h) に変更する
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        # qk^T を計算するために kを転地する
        k_t = k.transpose(2, 3)

        # qk^T
        # (B, h, N, D/h) x (B, h, D/h, N) -> (B, h, N, N)
        dots = (q @ k_t) / self.sqrt_dh
        
        # softmax
        attn = F.softmax(dots, dim=-1)

        # dropout
        attn = self.attn_drop(attn)

        # 加重和 softmax(qt^t/sqrt(Dh))v
        ## (B, h, N, N) x (B, h, N, D/h) -> (B, h, N, D/h)
        out = attn @ v

        # transpose
        ## (B, h, N, D/h) -> (B, N, h, D/h)
        out = out.transpose(1, 2)

        ## (B, N, h, D/h) -> (B, N, D)
        out = out.reshape(batch_num, patch_num, self.emb_dim)

        # Output
        # (B, N, D) -> (B, N, D)
        out = self.w_o(out)

        return out

batch_size = 2
channel = 3
height = 32
weight = 32

x = torch.randn(batch_size, channel, height, weight)
input_layer = VitInputLayer(num_patch_row=2)
z_0 = input_layer(x)

mhsa = MultiHeadSelfAttention()
z_0 = mhsa(z_0)

